In [ ]:
print("Starting Spark POOL Session")
import cloudbillingtool

from cloudbillingtool import azure_billing
from cloudbillingtool import hetzner_billing

import pkg_resources
print( pkg_resources.get_distribution('cloudbillingtool').version )

In [ ]:
storage_account="senecstorage"
azure_data="abfss://azurebilling@"+storage_account+".dfs.core.windows.net/monthly/SenecBillingReport/*/*.csv"
hetzner_data="abfss://hetznerbilling@"+storage_account+".dfs.core.windows.net/*.csv"
mappingfiles="abfss://mappingfiles@"+storage_account+".dfs.core.windows.net"
output_dir="abfss://allbilling@"+storage_account+".dfs.core.windows.net"

In [ ]:
azure_billing_with_tags = azure_billing.load_files_with_mapping(spark, azure_data, mappingfiles)
hetzner_billing_with_tags = hetzner_billing.load_files_with_mapping(spark, hetzner_data, mappingfiles)

# combine azure with hetzner billing
all_billing = azure_billing_with_tags.rdd.union(hetzner_billing_with_tags.rdd)

In [ ]:
# For Debugging
#all_billing.toDF().show()
#all_billing_df = all_billing.toDF()
#all_billing_df.toPandas().head(100)

In [ ]:
all_billing_df = all_billing.toDF()
all_billing_df.write \
    .format("com.microsoft.kusto.spark.synapse.datasource") \
    .option("spark.synapse.linkedService", "AzureDataExplorerCloudBilling") \
    .option("kustoDatabase", "senecallbillingdb") \
    .option("kustoTable", "AllBilling") \
    .option("tableCreateOptions","CreateIfNotExist") \
    .mode("Append") \
    .save()

In [ ]:
from pyspark.sql.functions import explode
df_distinct_tags = all_billing_df.select( explode("CostResourceTag").alias("CostResourceTag") ).distinct()
    
df_distinct_tags.write \
    .format("com.microsoft.kusto.spark.synapse.datasource") \
    .option("spark.synapse.linkedService", "AzureDataExplorerCloudBilling") \
    .option("kustoDatabase", "senecallbillingdb") \
    .option("kustoTable", "CostResourceTags") \
    .option("tableCreateOptions","CreateIfNotExist") \
    .mode("Append") \
    .save()
